In [1]:
!pip install google-search-results
!pip install langchain_community

!pip install langchain_openai
!pip install langgraph

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=e3a10a5919f6226599a9237292d1bc2bc0e43e39fe505e4c48a0c14206d46c12
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have r

In [2]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

In [4]:
llm = ChatOpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3.1:free",
    temperature=0,
    streaming=True
)

In [5]:
# --- Agent State Definition ---
# This TypedDict defines the structure of our application's state.
# It's how data is passed between the nodes in our graph.
class AgentState(TypedDict):
    task: str
    plan: str
    code: str
    execution_result: str
    test_result: str
    report: str
    # This special key 'messages' is for managing conversational history if needed.
    messages: Annotated[List[str], operator.add]

--- Agent Nodes ---
Each agent is represented by a function (a "node" in the graph).
These functions take the current state, perform their action, and return a dictionary
with the updated state values.

In [6]:
def manager_agent(state: AgentState) -> dict:
    """
    Manager agent: Receives the initial task and creates a high-level plan.
    """
    print("---MANAGER---")
    prompt = ChatPromptTemplate.from_template(
        """
        You are a project manager. Your role is to understand a user's programming request
        and create a clear, simple plan for your team.

        User Request: {task}

        Based on this, create a two-step plan:
        1.  A clear instruction for the Developer agent to write the Python code.
        2.  A clear instruction for the Tester agent to test the written code.
        """
    )
    chain = prompt | llm
    result = chain.invoke({"task": state["task"]})
    print(f"Manager's Plan:\n{result.content}")
    return {"plan": result.content}


In [7]:

def developer_agent(state: AgentState) -> dict:
    """
    Developer agent: Writes Python code based on the manager's plan and ensures it's executable.
    """
    print("---DEVELOPER---")

    # Use a loop to allow the agent to self-correct on syntax errors
    attempts = 0
    max_attempts = 3
    while attempts < max_attempts:
        print(f"Attempt {attempts + 1} of {max_attempts}")
        prompt = ChatPromptTemplate.from_template(
            """
            You are a Python developer. Your task is to write clean, executable Python code.
            Only output the raw Python code, without any markdown formatting or explanations.

            Manager's Plan:
            {plan}

            Current Code (if any, for refinement):
            <code>
            {code}
            </code>

            Execution Error (if any):
            {error}

            Write the Python code for the user's request: {task}
            """
        )
        chain = prompt | llm
        result = chain.invoke({
            "plan": state["plan"],
            "task": state["task"],
            "code": state.get("code", ""),
            "error": state.get("execution_result", "")
        })

        code = result.content.strip().replace("```python", "").replace("```", "")
        print(f"Developer's Code:\n{code}")

        # Execute the code to check for syntax errors
        try:
            exec(code)
            print("Code executed successfully (no syntax errors).")
            return {"code": code, "execution_result": "Code has no syntax errors."}
        except Exception as e:
            print(f"Execution failed with error: {e}")
            # If execution fails, update state and retry
            state["code"] = code
            state["execution_result"] = str(e)
            attempts += 1

    print("Developer failed to produce valid code after multiple attempts.")
    return {"code": code, "execution_result": "Failed to write executable code."}

In [8]:
def tester_agent(state: AgentState) -> dict:
    """
    Tester agent: Executes the code and reports on its functionality.
    """
    print("---TESTER---")
    prompt = ChatPromptTemplate.from_template(
        """
        You are a Quality Assurance (QA) tester. Your job is to execute the given Python code
        and report whether it works as expected based on the original task.

        Original Task: {task}
        Manager's Plan: {plan}

        Python Code to Test:
        <code>
        {code}
        </code>

        Provide a brief report on the outcome. Did the code run? Did it produce the expected result?
        """
    )
    chain = prompt | llm
    print("Executing code for testing...")
    try:

        exec(state["code"])
        test_outcome = "Code executed successfully. The functionality appears to work as per the task."
        print(test_outcome)
    except Exception as e:
        test_outcome = f"Code execution failed during testing. Error: {e}"
        print(test_outcome)

    return {"test_result": test_outcome}

In [9]:
def reporter_agent(state: AgentState) -> dict:
    """
    Reporter agent: Compiles the final report summarizing the entire process.
    """
    print("---REPORTER---")
    prompt = ChatPromptTemplate.from_template(
        """
        You are a reporting agent. Your task is to create a final, comprehensive summary
        of the entire development process. Combine all the information provided into a
        clear and concise report.

        **Initial Task:**
        {task}

        **Manager's Plan:**
        {plan}

        **Final Python Code:**
        <code>
        {code}
        </code>

        **Developer's Log:**
        {execution_result}

        **Tester's Report:**
        {test_result}

        Please generate the final summary report.
        """
    )
    chain = prompt | llm
    result = chain.invoke(state)
    print(f"---FINAL REPORT---\n{result.content}")
    return {"report": result.content}

In [10]:
# --- Graph Definition ---
# This is where we define the workflow (the directed graph).

# 1. Initialize the StateGraph
workflow = StateGraph(AgentState)

# 2. Add nodes to the graph. Each node is an agent function.
workflow.add_node("manager", manager_agent)
workflow.add_node("developer", developer_agent)
workflow.add_node("tester", tester_agent)
workflow.add_node("reporter", reporter_agent)

# 3. Define the edges that connect the nodes, determining the flow.
workflow.add_edge("manager", "developer")
workflow.add_edge("developer", "tester")
workflow.add_edge("tester", "reporter")
workflow.add_edge("reporter", END) # The reporter is the final step.

# 4. Set the entry point for the graph.
workflow.set_entry_point("manager")

# 5. Compile the graph into a runnable application.
app = workflow.compile()



In [11]:
# --- Main Execution Block ---
if __name__ == "__main__":
    initial_task = input("Please enter the programming task for the agent team: ")

    # The input to the graph is a dictionary with the initial state.
    inputs = {"task": initial_task, "messages": []}

    # Invoke the graph and stream the output.
    # The output will be the final state of the graph after all nodes have run.
    for output in app.stream(inputs):
        # The key is the name of the node that just ran.
        # The value is the state dictionary after that node ran.
        for key, value in output.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value, sep="\n", end="\n---\n")

    # The final state can be accessed from the last event in the stream
    final_state = list(app.stream(inputs))[-1]
    final_report = final_state['reporter']['report']
    print("\n--- Summary ---")
    print(final_report)

Please enter the programming task for the agent team: write a python program to find factorial of a number
---MANAGER---


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1757808000000'}, 'provider_name': None}}, 'user_id': 'user_32dAnj40R5oqxmnUSeZzXntoqiP'}